In [1]:
import os
import datetime
import numpy as np
import pandas as pd
from sqlalchemy import create_engine

In [2]:
# 数据库地址：数据库放在上一级目录下
db_path = os.path.join(os.path.dirname(os.getcwd()), "data.db")
engine_path = "sqlite:///" + db_path
# 创建数据库引擎
engine = create_engine(engine_path)

### 获取数据

In [3]:
sql = """
select * from business
"""

df = pd.read_sql(sql,engine)

df.sample(5)

,index,user_id,create_time,order_id,amount
13166,13166,72797,2021/2/26,4137903132,59.0
62145,62145,104941,2021/10/19,4201557665,49.0
29403,29403,127175,2021/5/11,4192533398,59.0
67264,67264,147490,2021/11/15,4201144712,690.0
62477,62477,142531,2021/10/21,4202724341,3999.0


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 76050 entries, 0 to 76049
Data columns (total 5 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   index        76050 non-null  int64  
 1   user_id      76050 non-null  int64  
 2   create_time  76048 non-null  object 
 3   order_id     76050 non-null  int64  
 4   amount       76043 non-null  float64
dtypes: float64(1), int64(3), object(1)
memory usage: 2.9+ MB


In [5]:
# create_time和amount有缺失值，去掉缺失值
df2 = df.copy()

# dropna() 默认只要该行有 nan 值就删除
df2 = df2.dropna()

df2.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 76041 entries, 0 to 76049
Data columns (total 5 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   index        76041 non-null  int64  
 1   user_id      76041 non-null  int64  
 2   create_time  76041 non-null  object 
 3   order_id     76041 non-null  int64  
 4   amount       76041 non-null  float64
dtypes: float64(1), int64(3), object(1)
memory usage: 3.5+ MB


### 数据分析

#### Recent【最近一次消费数据】

In [6]:
df2.sample()

,index,user_id,create_time,order_id,amount
9768,9768,105374,2021/2/12,4177624931,299.0


In [7]:
now_ = pd.to_datetime(datetime.datetime.now())

In [8]:
# 添加时间差数据
df2["recent"] = df2["create_time"].map(lambda x:(now_-pd.to_datetime(x)).days)

df2.sample(5)

,index,user_id,create_time,order_id,amount,recent
50572,50572,85526,2021/8/22,4198432890,2499.0,136
52019,52019,155789,2021/8/29,4194429805,1699.0,129
34287,34287,147180,2021/6/4,4176346242,11.9,215
10736,10736,148400,2021/2/16,4190923658,399.0,323
68004,68004,87578,2021/11/18,4194488169,9.0,48


In [9]:
len(df2.user_id.unique())

55540

In [10]:
# 用户最近一次购买商品的时间
recent_df = df2.groupby(by="user_id",as_index=False).agg({"recent":"min"})

In [11]:
recent_df.sample(5)

,user_id,recent
49443,152385,214
21621,95355,232
36916,126707,139
4576,60465,26
34558,121836,70


#### Frequency【一段时间内的消费次数】

In [12]:
frequency_df = df2.groupby(by="user_id",as_index=False).agg({"order_id":"count"})

In [13]:
frequency_df.sort_values(by="order_id",ascending=False).head()

,user_id,order_id
11421,74270,12
41587,136108,6
15319,82389,6
21054,94248,6
34042,120846,6


#### Mount【一段时间内的消费总金额】

In [14]:
mount_df = df2.groupby(by="user_id",as_index=False).agg({"amount":"sum"})

In [15]:
mount_df.sort_values(by="amount",ascending=False).head()

,user_id,amount
11421,74270,89566.9
13524,78669,31010.9
51252,156095,18957.0
5773,62830,18164.0
41884,136682,17749.0


### RFM模型

In [16]:
# 根据 user_id 合并数据
rfm_df = recent_df.merge(
    frequency_df,on="user_id",how="left"
).merge(
    mount_df,on="user_id",how="left"
)

In [17]:
rfm_df.head()

,user_id,recent,order_id,amount
0,51220,107,1,129.0
1,51221,234,1,29.9
2,51224,165,2,12.9
3,51226,338,1,19.9
4,51227,258,1,9.9


In [18]:
rfm_df2 = rfm_df.copy()

In [19]:
# np.linspace(0,1)

In [20]:
rfm_df2["amount"].quantile(q=np.linspace(0,1,num=6),interpolation='nearest')

0.0        0.5
0.2       14.9
0.4       61.9
0.6      299.0
0.8     2286.0
1.0    89566.9
Name: amount, dtype: float64

#### 根据分位数分层

In [21]:
# 划分为 5 个等级
recent_labels = [5,4,3,2,1]
mount_labels = [1,2,3,4,5]

In [22]:
m_bins = rfm_df2["amount"].quantile(q=np.linspace(0,1,num=6),interpolation='nearest')
r_bins = rfm_df2["recent"].quantile(q=np.linspace(0,1,num=6),interpolation='nearest')

In [23]:
rfm_df2["R"] = pd.cut(rfm_df2["recent"],bins=r_bins,labels=recent_labels,include_lowest=True)

In [24]:
rfm_df2["M"] = pd.cut(rfm_df2["amount"],bins=m_bins,labels=mount_labels,include_lowest=True)

In [25]:
rfm_df2.head()

,user_id,recent,order_id,amount,R,M
0,51220,107,1,129.0,4,3
1,51221,234,1,29.9,2,2
2,51224,165,2,12.9,3,1
3,51226,338,1,19.9,1,2
4,51227,258,1,9.9,2,1


#### 自定义分层

In [26]:
np.arange(1,4)

array([1, 2, 3])

In [27]:
frequency_bins = [1,3,5,12]
frequency_labels = [1,2,3]

In [28]:
rfm_df2["F"] = pd.cut(
    rfm_df2["order_id"]
    ,bins=frequency_bins
    ,labels=frequency_labels
    ,include_lowest=True
)

In [29]:
rfm_df2.sample(5)

,user_id,recent,order_id,amount,R,M,F
6682,64717,181,1,318.0,3,4,1
18055,88118,312,1,11.9,1,1,1
4759,60823,252,1,79.0,2,3,1
5917,63134,295,2,2263.9,1,4,1
8322,67946,212,1,8.9,2,1,1


#### 打标签

In [30]:
rfm_df2 = rfm_df2.astype(int)

In [31]:
rfm_df2["RFM"] = rfm_df2["R"]*3+rfm_df2["F"]*2+rfm_df2["M"]*5

In [32]:
rfm_df2.sample(5)

,user_id,recent,order_id,amount,R,M,F,RFM
36490,125851,205,1,9,2,1,1,13
30893,114320,57,2,3408,5,5,1,42
17864,87696,151,2,1924,3,4,1,31
46014,145313,52,2,67,5,3,1,32
38263,129396,132,1,28,4,2,1,24


In [33]:
rfm_df2

,user_id,recent,order_id,amount,R,M,F,RFM
0,51220,107,1,129,4,3,1,29
1,51221,234,1,29,2,2,1,18
2,51224,165,2,12,3,1,1,16
3,51226,338,1,19,1,2,1,15
4,51227,258,1,9,2,1,1,13
...,...,...,...,...,...,...,...,...
55535,165092,194,1,8,3,1,1,16
55536,165095,104,1,16,4,2,1,24
55537,165096,347,1,499,1,4,1,25
55538,165098,148,1,10,3,1,1,16


In [34]:
rfm_bins =rfm_df2["RFM"].quantile(q=np.linspace(0,1,num=6),interpolation='nearest').unique()
rfm_labels = ['流失客户','一般维持客户','重要挽留客户','重要唤回客户','重要价值客户']

In [35]:
rfm_bins

array([10, 18, 23, 29, 34, 46])

In [36]:
rfm_df2["客户标签"] = pd.cut(
    rfm_df2["RFM"],
    bins=rfm_bins,
    labels=rfm_labels,
    include_lowest=True,
    duplicates="drop"
)

In [37]:
rfm_df2.head()

,user_id,recent,order_id,amount,R,M,F,RFM,客户标签
0,51220,107,1,129,4,3,1,29,重要挽留客户
1,51221,234,1,29,2,2,1,18,流失客户
2,51224,165,2,12,3,1,1,16,流失客户
3,51226,338,1,19,1,2,1,15,流失客户
4,51227,258,1,9,2,1,1,13,流失客户


In [38]:
rfm_df2["R"].value_counts()

4    11228
5    11164
2    11110
1    11079
3    10959
Name: R, dtype: int64

In [39]:
rfm_df2["F"].value_counts()

1    54974
2      560
3        6
Name: F, dtype: int64

In [40]:
rfm_df2["M"].value_counts()

3    11389
1    11146
2    11089
5    11021
4    10895
Name: M, dtype: int64

In [41]:
rfm_df2["客户标签"].value_counts()

重要挽留客户    12576
流失客户      12130
一般维持客户    10556
重要唤回客户    10455
重要价值客户     9823
Name: 客户标签, dtype: int64

In [42]:
i = rfm_df2["客户标签"].value_counts().index.tolist()

In [43]:
v = rfm_df2["客户标签"].value_counts().values.tolist()

In [44]:
# rfm_df2[rfm_df2["客户标签"]=="流失客户"].sample(10)

### 数据可视化

In [45]:
from pyecharts import options as opts
from pyecharts.charts import Pie
from pyecharts.faker import Faker


c = (
    Pie()
    .add(
        "",
        [list(z) for z in zip(i, v)],
        radius=["30%", "75%"],
    )
    .set_global_opts(
        title_opts=opts.TitleOpts(title="客户分层占比"),
        legend_opts=opts.LegendOpts(orient="vertical", pos_top="15%", pos_left="2%"),
    )
    .set_series_opts(label_opts=opts.LabelOpts(formatter="{b}: {d}%"))
)
c.render_notebook()

In [46]:
rfm_df2.groupby(by="客户标签",as_index=False).agg({"amount":"sum"})

,客户标签,amount
0,流失客户,228951
1,一般维持客户,789369
2,重要挽留客户,5285248
3,重要唤回客户,19547844
4,重要价值客户,33236897
